In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} pygraphviz

# Ülesande püstitus

Proovime lihtsamat ülesannet. Vaata kuidas verb + määrsõna teiste nimisõnadega kombineerub
    
     kirjutas lepingule alla 

Ka siin tekib sul palju jama, sest sul võivad olla aja ja koha määrused sees, aga  proovi eraldada andmeid kujul

kirjutas alla -->{lepingule:obl, jaanuaris:obl}
kirjutas alla -->{dokumedile:obl}

st jäta alles kõik lapstipud mis pole alus ja märgi ära ka vastav deprel annotatsioon

Äkki õnnestub selle peale vaadates  osa ebaolulistest määrustest välja lõigata ja see subcat seos sealt välja õngitseda  


## Täpsustus

Võtame sisse nimisõnad, asesõnad, pärisnimed ja nende käänded
Lisame info, kas verb oli eitusega (feat sisaldab neg väärtust). Kui on eitav, siis muudame verbi kujule "ei VERB"

## Vajalikud teegid

In [2]:
from __future__ import print_function

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from collections import defaultdict
import matplotlib.pyplot as plt
import random

#import etnltk
#from estnltk.storage.postgres import PostgresStorage

import networkx as nx
import sqlite3
import sys
import json

## Abifunktsioonid

In [3]:
class graphFunctions:

    # kahe listi ühisosa 
    def intersection(a, b):
        return list(set(a).intersection(b))

    # tipu leidmine atribuudi väärtuse järgi
    def get_nodes_by_attributes(G,  attrname, attrvalue ):
        nodes = defaultdict(list)
        {nodes[v].append(k) for k, v in nx.get_node_attributes(G,attrname).items()}
        if attrvalue in nodes:
            return dict(nodes)[attrvalue]
        return []

    # graafi joonistamine 
    # tipp - lemma
    # serv - deprel
    def drawGraph(G):
        pos = graphviz_layout(G, prog='dot')
        labels = nx.get_node_attributes(G, 'lemma')
        nx.draw(G, pos, cmap = plt.get_cmap('jet'),labels=labels, with_labels=True)
        edge_labels = nx.get_edge_attributes(G, 'deprel')
        nx.draw_networkx_edge_labels(G, pos, edge_labels)
        plt.show()


In [4]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

In [5]:
def getCase(arr):
    for attr in arr:
        if attr in ('nom' #nimetav
                    , 'gen' #omastav
                    , 'part' #osastav
                    , 'adit' #lyh sisse
                    , 'ill' #sisse
                    , 'in'#sees
                    , 'el' #seest
                    , 'ad' #alale
                    , 'all' #alal
                    , 'abl' #alalt
                    , 'tr' #saav
                    , 'term' #rajav
                    , 'es' #olev
                    , 'abes'  #ilma#
                    , 'kom' #kaasa#
                   ) :
            return attr

def getNumber(arr):
    for attr in arr:
        if attr in ('sg' #ainsus
                    , 'pl' #mitmus
                   ):
            return attr


### Baasi  (sqlite3)  ettevalmistamine täitmiseks

Luuakse tabelid:
* collections_processed - salvestatakse viimane salvestatud collection Id
* {TABLENAME} - tabel kollokatsioonidega
 * lemma1 text
 * pos1 text
 * lemma2 text
 * pos2 text
 * deprel text
 * children json
 * total integer
 * example1 text
 * example2 text
 * example3 text

Luuakse indeksid:
* collections_processed_uniq
* {TABLENAME}_col1_col2_unique

In [6]:
def prepCollDb():
    global TABLENAME, cursor, conn

    cursor.execute(f"""CREATE TABLE IF NOT EXISTS collections_processed
                      (tablename text, lastcollection integer);
                      """)
    
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS collections_processed_uniq ON collections_processed(tablename);
    """)
    
    #tsv failist lugemise korral loome tabeli alati nullist
    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, 0, 0,) )
    
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS all_in_one
                      (lemma1 text, pos1 text, lemma2 text, pos2 text, deprel text, children json, total integer, example1 text, example2 text, example3 text);
                      """)

    #cursor.execute(f"""CREATE TABLE IF NOT EXISTS collocations
    #                  (lemma1 text, pos1 text, lemma2 text, pos2 text, total integer, deprel);
    #                  """)

    #cursor.execute(f"""CREATE TABLE IF NOT EXISTS children
    #                  (lemma text, pos text, lemma_case text, case_lemma text, case_pos text, deprel text, total integer);
    #                  """)

    #cursor.execute(f"""CREATE TABLE IF NOT EXISTS collections
    #                  (coll_id integer, childrens text, total integer);
    #                  """)
        
    #tsv failist lugemise korral loome tabeli alati nullist
    #cursor.execute(f"""DELETE FROM collocations;""")
    #cursor.execute(f"""DELETE FROM children;""")
    #cursor.execute(f"""DELETE FROM collections;""")
    cursor.execute(f"""DELETE FROM all_in_one;""")
    
    INDEXNAME = f'col1_col2_unique'
    
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS {INDEXNAME}_all_in_one
        ON all_in_one(lemma1, pos1, lemma2, pos2, deprel, children);
        """)
    
    #cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS {INDEXNAME}_collocations
    #    ON collocations(lemma1, pos1, lemma2, pos2, deprel);
    #    """)
    
    #cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS {INDEXNAME}_children
    #    ON children(lemma, pos, lemma_case, case_lemma, case_pos, deprel);
    #    """)
    
    #cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS {INDEXNAME}_collections
    #    ON collections(coll_id, childrens);
    #    """)
   
    conn.commit()



Tulemuse salvestamine baasi (sqlite3)

In [7]:
def saveCollToDb(collocations, examples, lastcollection):
    global TABLENAME, cursor, conn
    sqlColls = []
    #print ('here', collections)
    
    for key in collections.keys():
        #print ('key', key)
        #save key in parents or update count, get rowid
        
        #lemma1 text, pos1 text, lemma2 text, pos2 text, total integer, deprel
     
        example1 = None
        example2 = None
        example3 = None

        if len(examples[key])>0:
            example1 = examples[key][0]
        if len(examples[key])>1:
            example2 = examples[key][1]
        if len(examples[key])>2:
            example3 = examples[key][2]
        
        lemma1 = key[0]
        pos1 = key[2]
        lemma2 = key[1]
        pos2 = key[3]
        deprel = key[4]
        children = key[5]
        
        sqlColls.append( (lemma1, pos1, lemma2,  pos2, deprel, children, collocations[key], example1, example2, example3 , collocations[key], example1, example2,example3,) )

        
    cursor.executemany(f"""
        INSERT INTO all_in_one VALUES (?,?,?,?,?,?,?,?,?,?)
            ON CONFLICT(lemma1, pos1, lemma2, pos2, deprel, children)
            DO UPDATE SET total=total+? 
            ,  example1= ?
            ,  example2= ?
            ,  example3= ?
            ;""", sqlColls)

    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, lastcollection, lastcollection,) )
    
           
   
    conn.commit()
    eprint(f'andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - {lastcollection}' )
    

###  sõltuvuste leidmine 

* Kõik kolmikud, kus 
    * ülemus on tipp1 sõnaliigiga V ja alluv on tipp2 D 
    * ülemusel tipp1 on nimisõna/asesõna/pärisnimi (S/P/H) alluv, selle deprel ja kääne 

#### Näide 1

tõmbama	V	varem	D	advmod

<img src="img/all_deprel_verb_advberb_01.png" width="600" />

###Näide 2

saama	V	välja	D	compound:prt

In [8]:
def extract_verb_adverb_custom_deprel(G, collocations, examples, sentence):
   
    
    #graphFunctions.drawGraph(G)
    #print (sentence)
    # lyhim tee tippude vahel
    path = nx.all_pairs_shortest_path_length(G)
    path_reversed = nx.all_pairs_shortest_path_length(G.reverse())

    # kauguste maatriksid
    dpath = {x[0]:x[1] for x in path}
    dpath_reversed = {x[0]:x[1] for x in path}

    #eprint('eraldame tipud vajalike parameetritega ')
    verbnodes = graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'V')
    
    adverbnodes = graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'D')
    #vb on olulised ka pärisnimed ning asesõnad?
    nouns = graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'S') + graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'H')+ graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'P')
    
    cases = graphFunctions.get_nodes_by_attributes(G, attrname = 'deprel', attrvalue = 'case')
    
    #print ('cases', cases)
    #liigume tegusõnade kaupa
    
    #tekitame kollokatsioonipaari sellise:
    #Isa hülgas Jimmy kohe pärast tema sündi .
    # hülgama -[ advmod ]-> kohe sellel on id ja count?
    # children: obj Jimmy H nom
    # children: obj sünd S com pärast D
        #-[ obj ]-> Jimmy nom

        #-[ obl ]-> sündi com -[ case ]-> pärast
    
    for verb in verbnodes:
        childadverbs = []
        childnouns = []
        for adverb in adverbnodes:
            #kui on vahetu alluv
            if adverb in dpath[verb] and dpath[verb][adverb]==1: childadverbs.append(adverb)
        for noun in nouns:
            #kui on vahetu alluv
            # subj j2tame v2lja
            # kas siin peab ka nsubj:coop ja ssubj:coop olema?
            if G.nodes[noun]['deprel'] in ['nsubj', 'csubj']: continue
            if noun in dpath[verb] and dpath[verb][noun]==1: childnouns.append(noun)
        
        for adverb in childadverbs:
            
            #print ('collocation', key)
            children = []
            for noun in childnouns:
                child = {
                        'lemma': G.nodes[noun]['lemma']
                        , 'pos': G.nodes[noun]['pos']
                        , 'case':getCase(G.nodes[noun]['feat'])
                        , 'number':getNumber(G.nodes[noun]['feat'])
                        , 'deprel': G.nodes[noun]['deprel']
                        , 'case_children':[]
                        }
                #print ("\t" '-[',G.nodes[noun]['deprel'],']->', G.nodes[noun]['text'], getCase(G.nodes[noun]['feat']))
                #print ()
                childcases = []
                for case in cases:
                    if case in dpath[noun] and dpath[noun][case]==1: 
                        child['case_children'].append( (G.nodes[case]['lemma'], G.nodes[case]['pos'], ) )
                        
                        
               
                    
                
                children.append(child)
           
            key = ( 'ei ' + G.nodes[verb]['lemma'] if 'neg' in G.nodes[verb]['feat']  else G.nodes[verb]['lemma'], G.nodes[adverb]['lemma'], G.nodes[verb]['pos'], G.nodes[adverb]['pos'], G.nodes[adverb]['deprel'], json.dumps(children) ,)

                #synFunctions.drawGraph(G);
                #key = (G.nodes[verb]['lemma'], G.nodes[adverb]['lemma'], G.nodes[verb]['pos'], G.nodes[adverb]['pos'], G.nodes[adverb]['deprel'], )
                
               
            if not key in collocations:
                collocations[key] = 0
            if not key in examples:
                examples[key] = []
            collocations[key] += 1
            examples[key].append(sentence)
            if len(examples[key]) > 3:
                del(examples[key][random.randint(0, 2)])
                
                #if len(child['case_children'])>1: 
                #    print ('many cases',   key)
                #    graphFunctions.drawGraph(G)
            
    #print (collections)
    return (collocations, examples)



## Muutujad

**Korpuse TSV fail**

Lähte TSV-faili ja DB tabeli nimi, kuhu tulemus salvestatakse (TSV genereerimise kood on failis **collect_texts_db_tsv.ipynb**)

In [10]:
collectionName =  'koondkorpus_base_v2_20220216' 
#collectionName =  'koondkorpus_base_subset_of_5000_v2' 
corporafile = f'{collectionName}.tsv'
corporafile = '/Volumes/Selena/Kollokatsioonid/koondkorpus/koondkorpus_base_v2_20220216.tsv'


**Kollokatsiooni tüüp**

In [11]:
TYPE = 'kadrile_verb_adverb_noun_deprel'

**Tabeli nimi andmebaasis**

In [12]:
TABLENAME = f'{TYPE}_{collectionName}'

**Kollektsioonide arv**, mille kaupa salvestatakse vahepealne tulemus andmebaasi.

In [14]:
BATCHSIZE = 6000

## Andmebaasi loomine ja ette valmistamine

In [15]:
conn = sqlite3.connect(f"{TYPE}_collocations.db") #
cursor = conn.cursor()
prepCollDb()

## Kollokatsioonide leidmine

Andmed loetakse TSV failist. Igast lausest tehakse *networkx* suunatud graaf. Graafi servadeks on süntaksipuu 
head->child seos. Tippudeks on lause sõnad. 
Tipu omadused on:
* *id* - sõna id
* deprel
* lemma
* pos


In [16]:

unsaved = 0
collocations = {}
examples = {}
word_id = 0
count = 0
current_sentence = []

#
G = None
colId = None
prevCol = 0


with open(corporafile) as f:
    for line in f:
        count +=1
        line = line.strip("\n")
        row = line.split('\t')
        if not len(row) == 9:
            continue
        data = {}
        prevCol = colId
        (colId, data['start'], data['id'], data['text'], data['lemma'], data['upostag'], data['deprel'], data['head'], data['feat']) = row
        colId = int(colId)
        for k in ('id', 'start', 'head'):
            data[k] = int(data[k])

        word_id +=1
        #sentence start
        if data['id'] == 1:
            if not G==None:
                current_sentence_text = ' '.join([s['text'] for s in current_sentence])
                (collections, examples) = extract_verb_adverb_custom_deprel(G, collocations, examples, current_sentence_text)
                unsaved = 1
            current_sentence = []
            #count+=1
            word_id +=1
            G = nx.DiGraph()
            if not prevCol ==colId and not colId==0 and not colId%BATCHSIZE:
                saveCollToDb(collocations, examples, colId)
                collocations = {}
                unsaved = 0


        #paneme graafi kokku
        G.add_node(word_id, id=data['id'], text=data['text'],  lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'], feat=data['feat'].split('||'))
        G.add_edge(word_id - data['id'] + data['head'], word_id, deprel = data['deprel'])
        current_sentence.append(data)
        
        
    
    #viimane lause
    current_sentence_text = ' '.join([s['text'] for s in current_sentence])
    (collocations, examples) = extract_verb_adverb_custom_deprel(G, collocations, examples, current_sentence_text)
    saveCollToDb(collocations, examples, colId)


andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 6000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 12000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 18000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 24000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 30000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 36000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 42000


KeyboardInterrupt: 

### Andmebaasi indeksite lisamine


Tehakse kõige viimasena, et andmete sisestamine andmebaasi oleks kiirem.

In [17]:

indexesQ = [
    f'CREATE INDEX IF NOT EXISTS "deprel" ON all_in_one ("deprel");'
    , f'CREATE INDEX IF NOT EXISTS "lemma1" ON all_in_one ("lemma1");'
    , f'CREATE INDEX IF NOT EXISTS "lemma2" ON all_in_one("lemma2");'
    , f'CREATE INDEX IF NOT EXISTS "pos1" ON all_in_one ("pos1");'
    , f'CREATE INDEX IF NOT EXISTS "pos2" ON all_in_one ("pos2");'
    , f'CREATE INDEX IF NOT EXISTS "total" ON all_in_one ("total" DESC);']

for q in indexesQ: cursor.execute(q)

In [18]:
cursor.execute(f"SELECT count(*) FROM all_in_one")
all_collocations = cursor.fetchall()

In [19]:
all_collocations

[(600148,)]